In [1]:
# imports
import random

import pandas as pd
import numpy as np

from helpers import *

In [2]:
ACTIVITY_TRAIN = pd.read_csv('./activity_train.csv', names=['uniprot_id', 'mol_id', 'activity'])
ACTIVITY_TRAIN["mol_id"] = ACTIVITY_TRAIN["mol_id"].apply(remove_blank_space)


In [3]:
def RowCenterMatrix(M):
    V  = M.T
    mat= np.nanmean(V, axis=0)
    VC = V - mat
    VC[np.isnan(VC)]=0
    return VC.T

def create_rankingMatrix(rowLabel, colLabel, df):
    rows = list(set(ACTIVITY_TRAIN[rowLabel])) 
    cols = list(map(lambda v: v.strip(), set(ACTIVITY_TRAIN[colLabel])))
    n_rows = len(rows)
    n_cols = len(cols)

    rows = dict(zip(rows, np.arange(n_rows)))
    cols = dict(zip(cols, np.arange(n_cols)))
    mat = np.zeros((n_rows, n_cols))
    # mat[mat==0]=np.nan
    for rw in df.values:
        mat[rows[rw[0]], cols[rw[1].strip()]]=rw[2]
    return mat, rows, cols


In [4]:
mat, prots, mols=create_rankingMatrix("uniprot_id", "mol_id", ACTIVITY_TRAIN)
mat_centered = RowCenterMatrix(mat)
# pd.DataFrame(mat_centered, index=prots.keys(), columns=mols.keys())

In [5]:
def CosSim_SingleRow(M, row_index):
    # Extraia a linha especificada
    target_row = M[row_index, :]
    
    # Calcule as normas de todas as linhas
    norms = np.sqrt(np.sum(M * M, axis=1))
    norms[norms < 0.001] = 0.001  # Solução para linhas ou colunas sem variância
    
    # Calcule a norma da linha alvo
    target_norm = norms[row_index]
    
    # Calcule o produto escalar da linha alvo com todas as outras linhas
    dot_products = np.dot(M, target_row)
    
    # Calcule a similaridade coseno dividindo pelo produto das normas
    cos_similarities = dot_products / (norms * target_norm)
    
    return cos_similarities

In [6]:
df = pd.DataFrame(CosSim_SingleRow(mat_centered, 0), index=prots.keys(), columns=['P20309'])

In [7]:
df_filtrado = df[df['P20309'] > 0.5]

In [8]:
list(df_filtrado.index.values)

['P41145', 'P41143', 'P35372']